# Results Analysis

The goal of this notebook is to compare the text generations when using the traditional RAG and when usign the re-ranking with datamodels


In [11]:
import torch
import json
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from dmcr.evaluators.Rouge_L_evaluator import Rouge_L_evaluator

## Load Data

In [12]:
GENERATIONS_PATH = "generations"
rag_generations = json.load(open(f"{GENERATIONS_PATH}/generations.json"))
datamodels_generations = json.load(open(f"{GENERATIONS_PATH}/datamodels_generations.json"))

In [3]:
questions = pl.read_ipc("questions.feather")

In [18]:
np.array([datamodels_generations[str(0)][0]])

array(['According to the Death Penalty Information Center, there are'],
      dtype='<U60')

In [21]:
np.array([res])

array(['seven'], dtype='<U5')

In [27]:
datamodels_generations[str(1)]

[2,
 'There are 7 episodes in Big Little Lies Season',
 'Big Little Lies Season 2 has 7 episodes',
 'There are 7 episodes in Big Little Lies Season',
 'Big Little Lies Season 2 has 7 episodes']

In [ ]:
results = {
    "idx": [],
    "mean_metric_rag": [],
    "mean_metric_datamodels": [],
}
evaluator = Rouge_L_evaluator()


for i in range(len(rag_generations)):
    rag_i_results = []
    datamodels_i_results = []
    results["idx"].append(i)
    for j in range(len(rag_generations[str(i)])):

        max_rag = 0
        max_datamodels = 0
        
        for res in  questions[i]["answers"].to_numpy().flatten()[0]:

            metric_rag = evaluator.evaluate(np.array([res]), np.array([str(rag_generations[str(i)][j])]))
            metric_datamodels = evaluator.evaluate(np.array([res]), np.array([str(datamodels_generations[str(i)][j])]))
            
            max_rag = max(max_rag, metric_rag[0])
            max_datamodels = max(max_datamodels, metric_datamodels[0])

        rag_i_results.append(max_rag)
        datamodels_i_results.append(max_datamodels)

    results["mean_metric_rag"].append(np.mean(rag_i_results))
    results["mean_metric_datamodels"].append(np.mean(datamodels_i_results))
    df_results = pl.DataFrame(results)
    df_results.write_ipc("results.feather")

df_results = pl.DataFrame(results)

Pair: 2,718 - I couldn't find any information related to the total
Pair: 2,718 - I couldn't find any information related to the question
Pair: 2,718 - According to the FBI's data, there are approximately
Pair: 2,718 - I couldn't find any information related to the total
Pair: 2,718 - I don't have any information about death row inmates
Pair: seven - There are 7 episodes in Big Little Lies Season
Pair: seven - There are 7 episodes in Season 2 of
Pair: seven - Big Little Lies Season 2 has 7 episodes
Pair: seven - There are 7 episodes in Big Little Lies Season
Pair: seven - Big Little Lies Season 2 has 7 episodes
Pair: Foreigner - Sam Bailey
Pair: Foreigner - Sam Bailey
Pair: Foreigner - Sam Bailey
Pair: Foreigner - Sam Bailey
Pair: Foreigner - Sam Bailey
Pair: Saltfjellet - The Norwegian island of Kapan.


KeyboardInterrupt: 

In [8]:
rag_generations[str(i)][j]

'According to the Death Penalty Information Center, there are'

In [9]:
np.array([res])

array(['2,718'], dtype='<U5')